<a href="https://colab.research.google.com/github/LuizPradoJr/dio-lab-open-source/blob/main/C%C3%B3pia_de_SantanderDevWeek2023.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Santander Dev Week 2023 (ETL com Python)

**Contexto:** Você é um cientista de dados no Santander e recebeu a tarefa de envolver seus clientes de maneira mais personalizada. Seu objetivo é usar o poder da IA Generativa para criar mensagens de marketing personalizadas que serão entregues a cada cliente.

**Condições do Problema:**

1. Você recebeu uma planilha simples, em formato CSV ('SDW2023.csv'), com uma lista de IDs de usuário do banco:
  ```
  UserID
  1
  2
  3
  4
  5
  ```
2. Seu trabalho é consumir o endpoint `GET https://sdw-2023-prd.up.railway.app/users/{id}` (API da Santander Dev Week 2023) para obter os dados de cada cliente.
3. Depois de obter os dados dos clientes, você vai usar a API do ChatGPT (OpenAI) para gerar uma mensagem de marketing personalizada para cada cliente. Essa mensagem deve enfatizar a importância dos investimentos.
4. Uma vez que a mensagem para cada cliente esteja pronta, você vai enviar essas informações de volta para a API, atualizando a lista de "news" de cada usuário usando o endpoint `PUT https://sdw-2023-prd.up.railway.app/users/{id}`.



In [ ]:
# Utilize sua própria URL se quiser ;)
# Repositório da API: https://github.com/digitalinnovationone/santander-dev-week-2023-api
sdw2023_api_url = 'https://sdw-2023-prd.up.railway.app'

## **E**xtract

Extraia a lista de IDs de usuário a partir do arquivo CSV. Para cada ID, faça uma requisição GET para obter os dados do usuário correspondente.

In [ ]:
import pandas as pd

df = pd.read_csv('SDW2023.csv')
user_ids = df['UserID'].tolist()
print(user_ids)

[2781, 2782, 2783, 2784, 2785]


In [ ]:
import requests
import json

def get_user(id):
  response = requests.get(f'{sdw2023_api_url}/users/{id}')
  return response.json() if response.status_code == 200 else None

users = [user for id in user_ids if (user := get_user(id)) is not None]
print(json.dumps(users, indent=2))

[
  {
    "id": 2781,
    "name": "Cliente DIO 1",
    "account": {
      "id": 2937,
      "number": "991111",
      "agency": "991111",
      "balance": 2000.0,
      "limit": 3000.0
    },
    "card": {
      "id": 2700,
      "number": "**** **** **** 991111",
      "limit": 2500.0
    },
    "features": [],
    "news": [
      {
        "id": 5601,
        "icon": "https://digitalinnovationone.github.io/santander-dev-week-2023-api/icons/credit.svg",
        "description": "**Invista para alcan\u00e7ar seus objetivos.**\n\nEssa frase \u00e9 curta e objeti"
      }
    ]
  },
  {
    "id": 2782,
    "name": "Cliente DIO 2",
    "account": {
      "id": 2938,
      "number": "992222",
      "agency": "992222",
      "balance": 2000.0,
      "limit": 3000.0
    },
    "card": {
      "id": 2701,
      "number": "**** **** **** 992222",
      "limit": 2500.0
    },
    "features": [],
    "news": [
      {
        "id": 5602,
        "icon": "https://digitalinnovationone.github.io/sant

## **T**ransform

Utilize a API do OpenAI GPT-4 para gerar uma mensagem de marketing personalizada para cada usuário.

In [ ]:
pip install bardapi

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.3/42.3 kB 1.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.7/75.7 kB 4.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 981.5/981.5 kB 8.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.0/76.0 kB 6.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.5/57.5 kB 5.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 14.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 23.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 6.0 MB/s eta 0:00:00
  Created wheel for langdetect: filename=langdetect-1.0.9-py3-none-any.whl size=993224 sha256=0e4eac834a4cc758f72c8726453874eb1238aa0c683876515369e98932ff4997
  Stored in directory: /root/.cache/pip/wheels/95/03/7d/59ea870c70ce4e5a370638b5462a7711ab78fba2f655d05106
Successfully built langdetect


In [ ]:
from bardapi import Bard
import os
os.environ['_BARD_API_KEY']="chave."

for user in users:
  truncated_news = news[:70]
  user['news'].clear
  user['news'].append({
      "icon": "https://digitalinnovationone.github.io/santander-dev-week-2023-api/icons/credit.svg",
      "description": truncated_news
  })

In [ ]:
from bardapi import Bard
import os
def generate_ai_news(user):
  input_text = f"Crie uma unica frase para {user['name']} sobre a importância dos investimentos financeiros maximo 99 caracteres"
  bard_output = Bard().get_answer(input_text)['content']
  return bard_output

for user in users:
  news = generate_ai_news(user)
  truncated_news = news[:70]
  print(news)
  user['news'].append({
      "icon": "https://digitalinnovationone.github.io/santander-dev-week-2023-api/icons/credit.svg",
      "description": truncated_news
  })

**Frase para Cliente DIO 1:**

**Invista seu dinheiro para conquistar seus sonhos.**

Essa frase é simples e direta, mas transmite uma mensagem poderosa: investir é o caminho para alcançar seus objetivos financeiros. Com o dinheiro investido, você pode construir patrimônio, ter uma aposentadoria tranquila, ou realizar outros sonhos.

A frase tem 97 caracteres, dentro do limite de 99 caracteres.

**Outra opção:**

**Invista seu dinheiro para ter mais liberdade financeira.**

Essa frase também é simples e direta, mas enfatiza a importância da liberdade financeira. Com o dinheiro investido, você não precisa depender de um salário para viver, podendo ter mais autonomia e flexibilidade em sua vida.

Essa frase tem 98 caracteres, dentro do limite de 99 caracteres.

Qual das frases você prefere?
**Invista para conquistar seus objetivos.** (95 caracteres)

Esta frase é concisa e direta, e transmite a mensagem de que os investimentos financeiros são uma ferramenta importante para alcançar objet

## **L**oad

Atualize a lista de "news" de cada usuário na API com a nova mensagem gerada.

In [ ]:
def update_user(user):
  response = requests.put(f"{sdw2023_api_url}/users/{user['id']}", json=user)
  return True if response.status_code == 200 else False

for user in users:
  success = update_user(user)
  print(f"User {user['name']} updated? {success}!")

User Cliente DIO 1 updated? True!
User Cliente DIO 2 updated? True!
User Cliente DIO 3 updated? True!
User Cliente DIO 4 updated? True!
User Cliente DIO 5 updated? True!
